TCC>TTC pulse timing
==

In [ ]:
%matplotlib inline
from mushi import kSFS
from histories import eta
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import msprime
import stdpopsim

In [ ]:
# plt.style.use('dark_background')

In [ ]:
# set this to e.g. your Downloads folder path if you want plots saved to pdfs
plot_dir = None

In this exercise, you'll be jointly inferring demographic history and mutation spectrum history in the five European subpopulations of the 1000 Genomes data. These groups are: 
    CEU (individuals of European descent from Utah sampled by the Centre d'Etude Humaine)
    FIN (Finnish)
    GBR (Great Britain)
    IBS (Spanish from the Iberian Penninsula)
    TSI (Italians from Tuscany)

In [ ]:
pops = {'CEU', 'FIN', 'GBR', 'IBS', 'TSI'}

First, we'll load a "3-SFS" for each population, meaning the site frequency spectrum partitioned by triplet mutation context, and plot the total SFS as follows:

In [ ]:
ksfs_dict = {}
plt.figure(figsize=(3, 3))
for pop in pops:
    ksfs_df = pd.read_csv(f'example_data/{pop}/3-SFS.tsv', sep='\t', index_col=0)
    ksfs_dict[pop] = kSFS(X=ksfs_df.values, mutation_types=ksfs_df.columns)
    ksfs_dict[pop].plot_total(kwargs=dict(ls='', marker='o', ms=5, alpha=0.75, label=pop))
plt.legend()
plt.show()

In the absence of strong selective sweeps, site frequency spectra should be monotonic, i.e. allele abundance is a strictly decreasing function of allele frequency. In real data, however, there is almost always a smile-like uptick where allele abundance increases with allele frequency as the frequency gets close to 1. This is an artifact of infinite sites model violations where multiple mutations occur at the same site along the human and chimpanzee lineages, causing very low frequency mutations to be misclassified as very high frequency mutations. (Since low frequency mutations are so much more abundant than high frequency mutations, even a small number of such errors tend to outnumber true variants that are almost but not quite fixed in the population.) We will trim away (n-i)-tons for i ranging from 0 to 10 to prevent these problematic high frequency categories from throwing off our demographic inference.

In [ ]:
clip_low = 0
clip_high = 10
# we need a different mask vector for each population becuase the number of haplotypes n
# (length of SFS vector) varies
freq_mask = {}
for pop in pops:
    freq_mask[pop] = np.array([True if (clip_low <= i < ksfs_dict[pop].n - clip_high - 1) else False
                               for i in range(ksfs_dict[pop].n - 1)])

Need to set a grid of time discretization epoch boundaries (measured in generations):

In [ ]:
change_points = np.logspace(np.log10(1), np.log10(200000), 200)

Compute the masked genome size (excluding conserved sites, repeats, 1KG stict mask, and uncertain ancestral states), which is the subset of the genome where mutations could potentially be observed:

In [ ]:
with open('example_data/masked_size.tsv') as f:
    masked_genome_size = int(f.read())

Set the mutation rate per site per generation (here using a modern mutation rate estimated from trio data):

In [ ]:
u = 1.3e-8

Compute the expected number of mutations per masked genome per generation

In [ ]:
mu0 = u * masked_genome_size

Set the human generation time (in years) for time calibration

In [ ]:
t_gen = 29

Specify some regularization paramaters and convergence criteria, which dictate how smooth mushi will force the solutions to be and how long to search for an optimal solution:

In [ ]:
regularization_eta = dict(alpha_tv=1e2, alpha_spline=5e3, alpha_ridge=1e-10)
regularization_mu = dict(hard=True, beta_rank=0, beta_tv=7e1, beta_ridge=1e-10)
convergence = dict(tol=1e-10, max_iter=1000)

Estimate each population's inverse coalescence rate, $\eta(t)$, proportional to the effective population size at time $t$:

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(8, 4))
for idx, pop in enumerate(pops):
    print(pop)
    # clear solutions, in case rerunning this cell
    ksfs_dict[pop].clear_eta()
    ksfs_dict[pop].clear_mu()
    ksfs_dict[pop].infer_history(change_points, mu0, infer_mu=False,
                                 loss='prf', **regularization_eta,
                                 **convergence, mask=freq_mask[pop])
    plt.sca(axes[0])
    ksfs_dict[pop].plot_total(kwargs=dict(label=pop, ls='', marker='o', ms=5, alpha=0.75, c=f'C{idx}'),
                    line_kwargs=dict(c=f'C{idx}', alpha=0.75, lw=3),
                    fill_kwargs=dict(color=f'C{idx}', alpha=0.1))    
    plt.sca(axes[1])
    ksfs_dict[pop].eta.plot(lw=3, label=pop, t_gen=t_gen)
    plt.xlim([1e3, 1e6])
    plt.legend()
if plot_dir:
    plt.savefig(f'{plot_dir}/europulse.eta.pdf')
plt.show()

Loop over a few values of the `beta_spline` parameter, which controls how much L2 smoothness is imposed on the first derivative, and fit $\mu(t)$, the vector of mutation rates in all triplet contexts as a function of time:

In [ ]:
beta_spline_array = (1e-20, 1e3, 1e4)
fig, axes = plt.subplots(len(beta_spline_array), 2,
                         sharex='col', figsize=(8, 3 * len(beta_spline_array)))
for idx, beta_spline in enumerate(beta_spline_array):
    print(f'beta_spline = {beta_spline}')
    for idx2, pop in enumerate(pops):
        print(pop)
        # clear solution, in case rerunning this cell
        ksfs_dict[pop].clear_mu()
        ksfs_dict[pop].infer_history(change_points, mu0, beta_spline=beta_spline, infer_eta=False,
                                     loss='prf', **regularization_mu,
                                     **convergence, mask=freq_mask[pop])
        plt.sca(axes[idx, 0])
        ksfs_dict[pop].plot('TCC>TTC', clr=True,
                            label=(pop if idx == 0 else None),
                            lw=3, alpha=0.5, c=f'C{idx2}')
        plt.xscale('log')
        if idx == 0:
            plt.legend()
        if idx < axes.shape[0] - 1:
            plt.xlabel(None)
        plt.sca(axes[idx, 1])
        ksfs_dict[pop].μ.plot(('TCC>TTC',), t_gen=t_gen, clr=True,
                              label=(pop if idx == 0 else None),
                              lw=3, alpha=0.5)
        plt.xscale('log')
        if idx < axes.shape[0] - 1:
            plt.xlabel(None)        
        plt.xlim([1e3, 1e6])
plt.tight_layout()
if plot_dir:
    plt.savefig(f'{plot_dir}/europulse.mu.pdf', dpi=300)
plt.show()

These plots each show a mutation pulse dominated by the C>T transitions in the TCC context. They differ in the smoothness being imposed upon the mutation rate vector $\mu(t)$, and you'll notice that the fit of the model to the data is slightly worse for the smoothest history than for the histories with sharper transitions.

If you have a good memory of lecture, you might notice something weird here: the pulse ends around 1-2 kya, as reported in Harris and Pritchard (2017), but extends much further into the past than 15 kya. We believe this longer pulse inferred by mushi is probably more accurate than this older pulse timing that was inferred by less sophisticated means. 

The old mutation pulse timing was not inferred jointly with European demography, but was instead inferred assuming a published demographic history to be the correct one. That demographic history, published by Tennessen et al. in 2012, is implemented in the "standard population simulation" package `stdpopsim`. We can use stdpopsim to plot the Tennessen et al. population history as a function of time:

In [ ]:
species = stdpopsim.get_species("HomSap")
model = species.get_demographic_model("OutOfAfrica_2T12")
ddb = model.get_demography_debugger()
steps = np.concatenate((np.array([0]), change_points))
eta_Tennessen = eta(change_points,
                1 / ddb.coalescence_rate_trajectory(steps=steps,
                                                    num_samples=[0, 2],
                                                    double_step_validation=False)[0])
plt.figure(figsize=(4, 4))
eta_Tennessen.plot(t_gen=t_gen, label='EUR (Tennessen et al.)')
plt.xlim([1e3, 1e6])
plt.legend()
plt.show()

Now, we can infer $\mu(t)$ assuming the above demography instead of the one mushi fit to the data:

In [ ]:
fig, axes = plt.subplots(1, 3,
                         sharex='col', figsize=(12, 4))
for idx, pop in enumerate(pops):
    print(pop)
    ksfs_dict[pop].clear_mu()
    ksfs_dict[pop].infer_history(change_points, mu0, eta=eta_Tennessen, beta_spline=1e-20,
                                 loss='prf', **regularization_mu, **convergence,
                                 mask=freq_mask[pop])
    plt.sca(axes[0])
    ksfs_dict[pop].plot_total(kwargs=dict(label=pop, ls='', marker='o', ms=5, alpha=0.75, c=f'C{idx}'),
                              line_kwargs=dict(c=f'C{idx}', alpha=0.75, lw=3),
                              fill_kwargs=dict(color=f'C{idx}', alpha=0.1)) 
    plt.sca(axes[1])
    ksfs_dict[pop].plot('TCC>TTC', clr=True, label=pop, lw=3, alpha=0.5, c=f'C{idx}')
    plt.sca(axes[2])
    ksfs_dict[pop].mu.plot(('TCC>TTC',), t_gen=t_gen, clr=True,
                          label=pop,
                          lw=3, alpha=0.5)
axes[0].legend()
axes[2].set_xlim([1e3, 1e6])
plt.tight_layout()
if plot_dir:
    plt.savefig(f'{plot_dir}/europulse.Tennessen.pdf', dpi=300)
plt.show()

It fits the total SFS quite poorly, and timing of the TCC pulse seems to be incorrectly scaled as a result. The number of segregating variants observed does not match what's expected under this demography:

In [ ]:
plt.figure(figsize=(3, 3))
for pop in pops:
    plt.plot(ksfs_dict[pop].X[freq_mask[pop], :].sum(),
             (ksfs_dict[pop].L @ ksfs_dict['CEU'].mu.Z)[freq_mask[pop], :].sum(),
             'o', label=pop)
plt.plot([.8e7, 1.05e7], [.8e7, 1.05e7], '--k')
plt.xlabel('observed S')
plt.ylabel('predicted S')
plt.legend()
plt.show()

We believe that part of the reason for the poor fit of the Tennessen et al. model to the data is a cryptic assumption that the mutation rate was `~2.35e-8`, which was the standard rate in use before trio sequencing replaced phylogenetic calibration as the gold standard for human genetic research. Another contributing factor is likely that Tennessen, et al. did not fit their model to the Phase 3 1000 Genomes data, which were not published until 2015. 

Mushi reveals that attempts to infer the ages of mutation spectrum changes are exquisitely sensitive to underlying assumptions about the demographic history. The Tennessen et al. model is far from a cartoon bottleneck model, and is widely used as a "gold standard" European demographic history. 

Mushi infers that the TCC pulse likely began closer to 100,000 years ago than 15,000 years ago, which is surprising given that European and East Asian population divergence was more recent than this. The resolution to this paradox may be European population structure. In other words, the ancient TCC pulse may have been spatially localized to a European subpopulation that did not exchange gene flow with East Asians prior to 100,000 years ago. 